In [ ]:
# Dependencies
import re
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from modules.words import load_data

%matplotlib inline

# Constants
years = [2018, 2019]

# Load tweets dataset table
tweets = load_data('greta')

[nltk_data] Downloading package wordnet to /home/laura/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/laura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Count occurrences of hashtags used as seeds\

In [ ]:
def count_words_in_row(row, words_dict):
    """ 
    Conta una sola volta eventuali hashtag ripetuti. Bug o ok?
    """
    # Find the set of hashtags present in the tweet
    text = row.text.lower().replace("#", " #")
    hash_tweet = set(re.findall(r"#(\w+)", text))
    hash_dict = set(words_dict.keys())
    # Update the number of hashtag in the dictionary
    for word in (hash_tweet & hash_dict):
        words_dict[word] += 1



def count_words_in_list(df, words_list):
    # Initialize the dictionary to store counts
    words_count = dict(zip(words_list, [0]*len(words_list)))
    # Scan each row updating the counters
    df.apply(count_words_in_row, words_dict=words_count, axis=1)
    
    return words_count

In [ ]:
# 1st line: neutral hashtags
# 2nd line: specific hastags
words_list=["climatechange", "climatecrisis", "parisagreement", 
            "gretathunberg", "climatestrike", "fridays4future"]
                    
datasets = [tweets[tweets.created_at.dt.year == years[0]],
            tweets[tweets.created_at.dt.year == years[1]]]
            
for i, data in enumerate(datasets):
    print('hashtag frequencies in {}: \n\n'.format(years[i]), count_words_in_list(data, words_list), end = '\n\n')

## Find occurrences of all the used hashtags

In [ ]:
def find_hash_distribution(row, d):
    # Find all the hashtag in the tweet
    hash_list = re.findall(r"#(\w+)", row.text.lower())
    
    # Increase the counter or define a new key with value 1
    for hashtag in hash_list:
        if hashtag in d: 
            d[hashtag] += 1
        else:
            d[hashtag] = 1



def plot_hash_distribution(df, n_hash_max, n_hash_min=0):
    # Initialize the dictionary to store counts
    hash_dict = {}
    # Scan each row of the df
    df.apply(find_hash_distribution, d=hash_dict, axis=1)
    
    # Convert the dictionary in a df, sort it by frequency
    hash_df = pd.DataFrame.from_dict(hash_dict.items())
    hash_df.columns = ["hashtag", "occurrences"]
    hash_df.sort_values(by="occurrences", ascending=False, inplace=True)

    # Plot the barplot with the specified interval of the most frequent hashtags
    fig, ax = plt.subplots(1,1,figsize=(20,5))
    hash_df[n_hash_min : n_hash_max].plot(x="hashtag", y="occurrences", kind="bar", ax = ax)
    plt.show()
    
    return hash_df



def define_mask(row, mask, words):
    hash_tweet = set(re.findall(r"#(\w+)", row.text.lower()))
    mask.append((hash_tweet & words) != set())

    

def filter_dataset(df, words, year, n_max=25, n_min=1):
    """
    Return a dataframe that contains tweets in which is present at least 
    one hashtags in list "words" written in the specified year.
    """
    mask = []
    if words:
        tweets.apply(define_mask, mask=mask, words=set(words), axis=1)
    else:
        mask = True
    
    count_words_in_list(tweets[mask & (tweets.created_at.dt.year == year)], words_list)
    top_hash = plot_hash_distribution(tweets[mask & (tweets.created_at.dt.year == 2019)], 
                       n_hash_max=n_max, n_hash_min=n_min)
    
    return top_hash

In [ ]:
top_hash_pre = filter_dataset(tweets, words=[], year=2018, n_max=25, n_min=1)

In [ ]:
top_hash_post = filter_dataset(tweets, words=[], year=2019, n_max=25, n_min=1)

In [ ]:
# top_hash_post[top_hash_post.hashtag.isin(words_list)]
top_hash_post.head()

In [ ]:
print(top_hash_pre.shape)
#print(top_hash_pre.hashtag.values.tolist())

In [ ]:
print(top_hash_post.shape)
#print(top_hash_post.hashtag.values.tolist())

In [ ]:
len(set(top_hash_post.hashtag.values.tolist()) - set(top_hash_pre.hashtag.values.tolist()))

In [ ]:
len(set(top_hash_pre.hashtag.values.tolist()) - set(top_hash_post.hashtag.values.tolist()))

## Consistency Check
### Verify if the output of a random line is coherent

In [ ]:
index = 0

text = tweets.loc[index].text
hash_list = re.findall(r"#(\w+)", text.lower())

print("TEXT: ", text, "\n\nHASHTAG LIST: ", hash_list)

### Compare with the original json

In [ ]:
import json_lines
import json

PATH_JSON = "data/postGreta.jsonl"

In [ ]:
def get_hashtags(tweet):
    """
    Parse tweet hashtags
    """
    return [t['text'].lower() for t in tweet['entities']['hashtags']]



def load_jsonl(file):
    with open(file, 'rb') as f:
        # extract relevant fields from tweets. Be aware that replies
        # have a different structure. For example, assuming we would
        # like to extract hashtags we need to distinguish between different cases
        # (other fields return truncated text and hastags)
        tweets_hashtags = {}
        for tweet in json_lines.reader(f, broken=True):
            # tweet is a reply
            if 'retweeted_status' in tweet:
                try:
                    final_tweet = tweet['retweeted_status']['extended_tweet']
                except:
                    # text and hashtags have not been truncated
                    final_tweet = tweet['retweeted_status']
            # no reply
            else:
                try:
                    final_tweet = tweet['extended_tweet']
                except:
                    final_tweet = tweet
            tweets_hashtags[tweet['id']] = get_hashtags(final_tweet)
    
    return tweets_hashtags

In [ ]:
hashtags_post = load_jsonl(PATH_JSON)

In [ ]:
import random

ghost_id = []
error_id = []

for i in range(1000):
    tweet_id = random.choice(list(hashtags_post.keys()))
    
    text = None
    try:
        text = tweets[tweets.id == str(tweet_id)].text.values[0]
    except:
        ghost_id.append(tweet_id)
        
    if text:
        hash_list_original = hashtags_post[tweet_id]
        hash_list_extracted = re.findall(r"#(\w+)", text.lower())
        
        if hash_list_original != hash_list_extracted:
            error_id.append(tweet_id)

In [ ]:
len(ghost_id)

In [ ]:
tweets[tweets.id == ghost_id[0]]